##  10 - Learning Policy on Image + Attributes - loss 2

Fully connected network with:
- Input: 
    - **184** size vector with:
        * client information: [current latitude, current longitud, destination latitude, destination longitude]
        * vehicles information: [load, queue, current latitude, current longitud, next latitude, next longitude]
    - Image with 31 channel: 
        * Channel 0 --> client representation. its current location is marked as 1; its destination as -1
        * Channels > 0 --> vehicle current location (1)
        
- Output: One hot encoding vector with 30 entries representing the available vehicles


The attribute input is merged later with a partial output of the network

loss 1: vehicle assignment
loss 2: vehicle position

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import pandas as pd
import random
import tqdm
import time

import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torch import optim

In [40]:
class Net_Final(nn.Module):
    def __init__(self, inp1, num_v, im_size, kernel):
        super().__init__()

        ins = 5
        self.cs = kernel
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(inp1, ins, kernel_size=self.cs),
            nn.BatchNorm2d(ins),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ins, 5, kernel_size=self.cs),
            nn.BatchNorm2d(5),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(2*(im_size - 3*(self.cs-1))*(im_size - 3*(self.cs-1)), 264),
            nn.ReLU(),
        
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(264, num_v),
        )
        
        self.f_aux = nn.Sequential(
            nn.Linear(num_v, 2)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(7*num_v, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
        )
        
        self.fc4 = nn.Sequential(
            nn.Linear(500, num_v),
            nn.BatchNorm1d(num_v),
            nn.ReLU(),
        )
        

    def forward(self, x, v_x):
        num_v = v_x.shape[1] 
        
        x0 = x
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        
        x4 = x3.view(-1, 2*(x.shape[-1] - 3*(self.cs-1))*(x.shape[-1] - 3*(self.cs-1)))
        x5 = self.fc1(x4)
        x6 = self.fc2(x5)
        
        x_aux = self.f_aux(x6)
        # add vehicles information
        x7 = torch.cat([v_x.transpose(2,1) ,x6.view(-1, 1,v_x.shape[1] )], dim=1).view(v_x.shape[0], -1)
        
        x8 = self.fc3(x7)
        x9 = self.fc4(x8)
        
        return x_aux, x9

In [12]:
def epoch_train(model, train_table, test_tables, optimizer, criterion1, criterion2, e, im_size, simple, weighted):
    
    #model.train()
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            
            y = np.hstack([data_y[i] for i in t_idx])
           
            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            train_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output1, output2 = model(train_x, train_x_aux)
            if simple:
                batch_loss = criterion2(output2, train_y)
            else:
                batch_loss = weighted*criterion1(output1, train_y_aux) + (1- weighted)*criterion2(output2, train_y)
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output2,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
    
    
    test_loss = 0
    test_acc = []
    
    #model.eval()
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            test_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            test_y = torch.tensor(y).type(torch.LongTensor)
            test_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            output1, output2 = model(test_x, test_x_aux)
            if simple:
                batch_loss = criterion2(output2, test_y)
            else:
                batch_loss = weighted*criterion1(output1, test_y_aux) + (1-weighted)*criterion2(output2, test_y)
            

            test_loss += batch_loss.item()
            _, a = torch.max(output2,1)
            
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures, output2
    

### General configurations

In [64]:
a = list(range(1,108))
b = a[:int(len(a)*0.6)]
c = a[int(len(a)*0.6):int(len(a)*0.9)]
d = a[int(len(a)*0.9):]

In [4]:
random.seed(4917)
global MAX_V, mini_batch_size, n_epochs, lr, inp_size
MAX_V = 30
mini_batch_size = 50
n_epochs=40

tables = list(range(1,108))
random.shuffle(tables)

lr = 0.0001
inp_size = 4 + MAX_V*6

train_tables, test_tables, validation_tables = \
            tables[:int(len(tables)*0.6)], tables[int(len(tables)*0.6):int(len(tables)*0.9)], \
            tables[int(len(tables)*0.9):]

### Functions

In [51]:
def evaluating_model(im_size, model, simple=False, weighted=0.5, n_epochs=50):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

### Net 1. - W=0.999; 500units; kernel size = 3

In [ ]:
total_acc, total_loss = [], []
kernel_size = 3
for im_size in [30, 50, 10]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = Net_Final(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.999)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))
#torch.save(model, 'model_try3.pt')

In [7]:
total_acc, total_loss = [], []
kernel_size = 3
for im_size in [50, 10]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = Net_Final(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.999)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 1.297 Accuracy: 0.858 Test Loss: 4.739 Accuracy: 0.252
Average time per epoch 29.022s +- 1.135
Max accuracy of 0.256 achieved at epoch 30
Epoch 40. Train Loss: 2.060 Accuracy: 0.854 Test Loss: 5.507 Accuracy: 0.256
Average time per epoch 28.594s +- 1.173
Max accuracy of 0.262 achieved at epoch 31
Epoch 40. Train Loss: 1.536 Accuracy: 0.851 Test Loss: 4.928 Accuracy: 0.252
Average time per epoch 28.033s +- 0.288
Max accuracy of 0.262 achieved at epoch 27
Epoch 40. Train Loss: 1.328 Accuracy: 0.860 Test Loss: 4.625 Accuracy: 0.255
Average time per epoch 28.025s +- 0.196
Max accuracy of 0.260 achieved at epoch 26
Epoch 40. Train Loss: 1.482 Accuracy: 0.855 Test Loss: 5.052 Accuracy: 0.253
Average time per epoch 28.073s +- 0.244
Max accuracy of 0.263 achieved at epoch 36
Average accuracy 0.261 +- 0.002. Av loss 4.241
 -------------
Epoch 40. Train Loss: 6.568 Accuracy: 0.794 Test Loss: 4.640 Accuracy: 0.241
Average time per epoch 3.352s +- 0.050
Max accuracy of 0.259 

In [55]:
model_size50 = Net_Final(31, MAX_V, 50, 3)
loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(50, model_size50, weighted=0.999, n_epochs=10)
torch.save(model_size50, 'model_size_50.pt')

Epoch 10. Train Loss: 4.349 Accuracy: 0.429 Test Loss: 4.468 Accuracy: 0.215
Average time per epoch 25.995s +- 0.982
Max accuracy of 0.215 achieved at epoch 9


In [57]:
model_size30 = Net_Final(31, MAX_V, 30, 3)
loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(30, model_size30, weighted=0.999, n_epochs=10)
torch.save(model_size30, 'model_size_30.pt')

Epoch 10. Train Loss: 5.237 Accuracy: 0.405 Test Loss: 4.512 Accuracy: 0.209
Average time per epoch 10.527s +- 0.920
Max accuracy of 0.211 achieved at epoch 8


## Different architectures

In [ ]:
print(model)

In [ ]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesP3(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

In [ ]:
model = ImagesP3(31, MAX_V, im_size, kernel_size)

In [ ]:
print(model)

In [ ]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesP3(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

## MODEL WITHOUT INTERMEDIATE LOSS

In [15]:
random.seed(4917)
global MAX_V, mini_batch_size, n_epochs, lr, inp_size
MAX_V = 30
mini_batch_size = 50
n_epochs=40

tables = list(range(1,98))
random.shuffle(tables)

lr = 0.0001
inp_size = 4 + MAX_V*6

train_tables, test_tables, validation_tables = \
            tables[:int(len(tables)*0.6)], tables[int(len(tables)*0.6):int(len(tables)*0.9)], \
            tables[int(len(tables)*0.9):]

In [16]:
total_acc, total_loss = [], []
kernel_size = 3
for im_size in [30, 50, 10]: # select im size based on previous results 
    main_acc, main_loss = [], []
    for trial in range(5):
        model = Net_Final(31, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.001)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 91.575 Accuracy: 0.935 Test Loss: 188.138 Accuracy: 0.1534
Average time per epoch 11.155s +- 1.115
Max accuracy of 0.186 achieved at epoch 4
Epoch 40. Train Loss: 93.962 Accuracy: 0.931 Test Loss: 186.689 Accuracy: 0.1561
Average time per epoch 10.902s +- 0.221
Max accuracy of 0.198 achieved at epoch 3
Epoch 40. Train Loss: 94.900 Accuracy: 0.931 Test Loss: 188.080 Accuracy: 0.1562
Average time per epoch 11.259s +- 0.636
Max accuracy of 0.184 achieved at epoch 3
Epoch 40. Train Loss: 89.395 Accuracy: 0.944 Test Loss: 190.905 Accuracy: 0.1340
Average time per epoch 11.936s +- 1.466
Max accuracy of 0.188 achieved at epoch 5
Epoch 40. Train Loss: 87.775 Accuracy: 0.943 Test Loss: 189.061 Accuracy: 0.1498
Average time per epoch 12.722s +- 1.200
Max accuracy of 0.185 achieved at epoch 7
Average accuracy 0.188 +- 0.005. Av loss 179.817
 -------------
Epoch 40. Train Loss: 75.894 Accuracy: 0.965 Test Loss: 190.135 Accuracy: 0.1394
Average time per epoch 28.978s +- 1.511


In [41]:
random.seed(4917)
global MAX_V, mini_batch_size, n_epochs, lr, inp_size
MAX_V = 30
mini_batch_size = 50
n_epochs=40

tables = list(range(1,98))
random.shuffle(tables)

lr = 0.0001
inp_size = 4 + MAX_V*6

train_tables, test_tables, validation_tables = \
            tables[:int(len(tables)*0.6)], tables[int(len(tables)*0.6):int(len(tables)*0.9)], \
            tables[int(len(tables)*0.9):]

In [44]:
def epoch_train(model, train_table, test_tables, optimizer, criterion1, criterion2, e, im_size, simple, weighted):
    
    #model.train()
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            
            y = np.hstack([data_y[i] for i in t_idx])
           
            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            train_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output1, output2 = model(train_x, train_x_aux)
            if simple:
                batch_loss = criterion2(output2, train_y)
            else:
                batch_loss = weighted*criterion1(output1, train_y_aux) + (1- weighted)*criterion2(output2, train_y)
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output2,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
    
    
    test_loss = 0
    test_acc = []
    
    #model.eval()
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][i][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            test_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            test_y = torch.tensor(y).type(torch.LongTensor)
            test_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            output1, output2 = model(test_x, test_x_aux)
            if simple:
                batch_loss = criterion2(output2, test_y)
            else:
                batch_loss = weighted*criterion1(output1, test_y_aux) + (1-weighted)*criterion2(output2, test_y)
            

            test_loss += batch_loss.item()
            _, a = torch.max(output2,1)
            
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures, output2
    

In [45]:
def evaluating_model(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(10):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

In [47]:
model = Net_Final(31, MAX_V, 10, 3)
loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(10, model, weighted=0.001)

Epoch 10. Train Loss: 284.910 Accuracy: 0.451 Test Loss: 177.152 Accuracy: 0.207
Average time per epoch 3.236s +- 0.199
Max accuracy of 0.207 achieved at epoch 8


In [50]:
torch.save(model, 'without_loss_weights.pt')

In [49]:
a = 1

## FUTURE LOCATION AS -1

In [17]:
class ImagesVal2(nn.Module):
    def __init__(self, inp1, num_v, im_size, kernel):
        super().__init__()

        ins = 5
        self.cs = kernel
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(inp1, ins, kernel_size=self.cs),
            nn.BatchNorm2d(ins),
            nn.ReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(ins, 5, kernel_size=self.cs),
            nn.BatchNorm2d(5),
            nn.ReLU(),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(5, 2, kernel_size=self.cs),
            nn.BatchNorm2d(2),
            nn.ReLU(),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(2*(im_size - 3*(self.cs-1))*(im_size - 3*(self.cs-1)), 264),
            nn.ReLU(),
        
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(264, num_v),
        )
        
        self.f_aux = nn.Sequential(
            nn.Linear(num_v, 2)
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(7*num_v, 300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
        )
        
        self.fc4 = nn.Sequential(
            nn.Linear(300, num_v),
            nn.BatchNorm1d(num_v),
            nn.ReLU(),
        )
        
        

    def forward(self, x, v_x):
        num_v = v_x.shape[1]
        
        x0 = x
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = x3.view(-1, 2*(x.shape[-1] - 3*(self.cs-1))*(x.shape[-1] - 3*(self.cs-1)))
        x5 = self.fc1(x4)
        x6 = self.fc2(x5)
        
        x_aux = self.f_aux(x6)
        # add vehicles information
        x7 = torch.cat([v_x.transpose(2,1) ,x6.view(-1, 1,v_x.shape[1] )], dim=1).view(v_x.shape[0], -1)
        
        x8 = self.fc3(x7)
        x9 = self.fc4(x8)
        
        return x_aux, x9

In [22]:
def epoch_train2(model, train_table, test_tables, optimizer, criterion1, criterion2, e, im_size=30, simple=False, weighted=0.5):
    
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv*2+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][2*i-1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                    x[k][2*i][int(data[cl][i][4]//dist)][int(data[cl][i][5]//dist)] = -1
            
            
            y = np.hstack([data_y[i] for i in t_idx])
           
            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            train_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output1, output2 = model(train_x, train_x_aux)
            if simple:
                batch_loss = criterion2(output2, train_y)
            else:
                batch_loss = weighted*criterion1(output1, train_y_aux) + (1- weighted)*criterion2(output2, train_y)
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output2,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
    
    
    test_loss = 0
    test_acc = []
    
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), nv*2+1, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][2*i -1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
                    x[k][2*i][int(data[cl][i][4]//dist)][int(data[cl][i][5]//dist)] = -1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            test_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            test_y = torch.tensor(y).type(torch.LongTensor)
            test_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            output1, output2 = model(test_x, test_x_aux)
            if simple:
                batch_loss = criterion2(output2, test_y)
            else:
                batch_loss = criterion1(output1, test_y_aux) + criterion2(output2, test_y)
            

            test_loss += batch_loss.item()
            _, a = torch.max(output2,1)
            
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures, output2

def evaluating_model(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train2(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

In [23]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesVal2(2*MAX_V+1, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 3.150 Accuracy: 0.766 Test Loss: 184.683 Accuracy: 0.180
Average time per epoch 16.338s +- 0.846
Max accuracy of 0.209 achieved at epoch 7
Epoch 40. Train Loss: 4.105 Accuracy: 0.760 Test Loss: 184.299 Accuracy: 0.184
Average time per epoch 16.223s +- 1.065
Max accuracy of 0.208 achieved at epoch 16
Epoch 40. Train Loss: 4.590 Accuracy: 0.746 Test Loss: 183.821 Accuracy: 0.178
Average time per epoch 16.311s +- 0.959
Max accuracy of 0.216 achieved at epoch 11
Epoch 40. Train Loss: 3.596 Accuracy: 0.772 Test Loss: 187.088 Accuracy: 0.174
Average time per epoch 16.201s +- 1.585
Max accuracy of 0.215 achieved at epoch 10
Epoch 40. Train Loss: 3.589 Accuracy: 0.765 Test Loss: 186.722 Accuracy: 0.173
Average time per epoch 15.540s +- 0.900
Max accuracy of 0.208 achieved at epoch 11
Average accuracy 0.211 +- 0.004. Av loss 180.079
 -------------


In [28]:
def evaluating_model(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(20):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train2(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

In [29]:
model = ImagesVal2(2*MAX_V+1, MAX_V, im_size, kernel_size)
loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model(im_size, model, weighted=0.99)

Epoch 20. Train Loss: 5.502 Accuracy: 0.544 Test Loss: 181.360 Accuracy: 0.200
Average time per epoch 15.509s +- 1.044
Max accuracy of 0.216 achieved at epoch 8


In [30]:
torch.save(model, 'clipped_weights.pt')

# -----------

## WITH CLIPPED VALUES IN IMAGES

In [33]:
def epoch_train3(model, train_table, test_tables, optimizer, criterion1, criterion2, e, im_size=30, simple=False, weighted=0.5):
    
    sum_loss = 0
    acc = []
    idx_failures = []
    dist = 1/(im_size-1)

    # train data
    for k, TABLE in enumerate(train_tables):
        
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()

        idx = list(data.keys())
        nv = len(data[idx[0]]) - 1
        
        

        for b in range(0, len(idx), mini_batch_size):
            # idx of clients to analyse
            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), 2, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            
            y = np.hstack([data_y[i] for i in t_idx])
           
            train_x = torch.tensor(x).type(torch.FloatTensor)
            train_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            train_y = torch.tensor(y).type(torch.LongTensor)
            train_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            # set gradient to zero
            optimizer.zero_grad()
            
            # compute output
            output1, output2 = model(train_x, train_x_aux)
            if simple:
                batch_loss = criterion2(output2, train_y)
            else:
                batch_loss = weighted*criterion1(output1, train_y_aux) + (1- weighted)*criterion2(output2, train_y)
            
            batch_loss.backward()
            optimizer.step()

            sum_loss = sum_loss + batch_loss.item()
            _, a = torch.max(output2,1)
            acc.append(float((train_y == a).sum())/len(train_y))
            
    
    
    test_loss = 0
    test_acc = []
    
    for k,TABLE in enumerate(test_tables):
        data = np.load('./minmax_data/data_vector_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        data_y = np.load('./minmax_data/data_vector_y_{}.npy'.format(TABLE), allow_pickle=True).tolist()
        
        idx = list(data.keys())
        #random.shuffle(idx)
        
        for b in range(0, len(idx), mini_batch_size):

            t_idx = idx[b:b+mini_batch_size]
            
            x = np.zeros((len(t_idx), 2, im_size, im_size))
            loc_y = np.zeros((len(t_idx), 2))
            x_aux = []
            
            for k, cl in enumerate(t_idx):
                loc = int(data_y[cl])
                loc_y[k] = [data[cl][loc+1][2], data[cl][loc+1][3]]
                
                x_aux.append(torch.tensor(np.asarray(data[cl][1:])).type(torch.FloatTensor))
                x[k][0][int(data[cl][0][0]//dist)][int(data[cl][0][1]//dist)] = 1
                x[k][0][int(data[cl][0][2]//dist)][int(data[cl][0][3]//dist)] = -1

                for i in range(1,31):
                    x[k][1][int(data[cl][i][2]//dist)][int(data[cl][i][3]//dist)] = 1
            
            y = np.hstack([data_y[i] for i in t_idx])
            
            
            test_x = torch.tensor(x).type(torch.FloatTensor)
            test_x_aux = torch.stack(x_aux).type(torch.FloatTensor)
            test_y = torch.tensor(y).type(torch.LongTensor)
            test_y_aux = torch.tensor(loc_y).type(torch.FloatTensor)
            
            output1, output2 = model(test_x, test_x_aux)
            if simple:
                batch_loss = criterion2(output2, test_y)
            else:
                batch_loss = criterion1(output1, test_y_aux) + criterion2(output2, test_y)
            

            test_loss += batch_loss.item()
            _, a = torch.max(output2,1)
            
            test_acc.append(float((test_y == a).sum())/len(test_y))
            idx_failures += [t_idx[i] for i in np.where(test_y != a)[0]]
            
            
    print('\rEpoch {}. Train Loss: {:.3f} Accuracy: {:.3f} Test Loss: {:.3f} Accuracy: {:.3f}'.format(e+1, sum_loss, np.mean(acc), test_loss,np.mean(test_acc)), end="")
    return sum_loss, np.sum(acc)/len(acc), test_loss, np.sum(test_acc)/len(test_acc), idx_failures, output2

def evaluating_model3(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(n_epochs):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train3(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

In [34]:
total_acc, total_loss = [], []
im_size=30
for kernel_size in [3]:
    main_acc, main_loss = [], []
    for trial in range(5):
        model = ImagesVal2(2, MAX_V, im_size, kernel_size)
        loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model3(im_size, model, weighted=0.99)
        main_loss.append(min(test_loss))
        main_acc.append(max(test_acc))
    total_acc.append(main_acc)
    total_loss.append(main_loss)
    print('Average accuracy {:.3f} +- {:.3f}. Av loss {:.3f}\n -------------'.format(\
        np.mean(main_acc), np.std(main_acc), np.mean(main_loss)))

Epoch 40. Train Loss: 4.269 Accuracy: 0.764 Test Loss: 187.326 Accuracy: 0.172
Average time per epoch 6.149s +- 0.467
Max accuracy of 0.198 achieved at epoch 14
Epoch 40. Train Loss: 3.888 Accuracy: 0.774 Test Loss: 185.226 Accuracy: 0.186
Average time per epoch 6.125s +- 0.214
Max accuracy of 0.215 achieved at epoch 12
Epoch 40. Train Loss: 3.925 Accuracy: 0.768 Test Loss: 186.414 Accuracy: 0.161
Average time per epoch 6.203s +- 0.374
Max accuracy of 0.205 achieved at epoch 13
Epoch 40. Train Loss: 3.932 Accuracy: 0.778 Test Loss: 184.613 Accuracy: 0.181
Average time per epoch 6.170s +- 0.250
Max accuracy of 0.206 achieved at epoch 11
Epoch 40. Train Loss: 4.150 Accuracy: 0.756 Test Loss: 185.620 Accuracy: 0.166
Average time per epoch 6.120s +- 0.203
Max accuracy of 0.206 achieved at epoch 11
Average accuracy 0.206 +- 0.005. Av loss 180.918
 -------------


In [35]:
def evaluating_model3(im_size, model, simple=False, weighted=0.5):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion2 = nn.CrossEntropyLoss() 
    criterion1 = nn.MSELoss() 
    
    loss, acc, test_loss, test_acc, idx_f, times = [], [], [], [], [], []

    for epoch in range(15):
        current_t = time.time()
        train_l, accuracy, test_l, test_a, idx_failures, o2 = \
        epoch_train3(model, train_tables, test_tables, optimizer, criterion1, criterion2, epoch, im_size, simple, weighted)
        
        times.append(time.time() - current_t)
        loss.append(train_l)
        test_loss.append(test_l)
        acc.append(accuracy)
        test_acc.append(test_a)
        idx_f.append(idx_failures)

    print('\nAverage time per epoch {:.3f}s +- {:.3f}'.format(np.mean(times), 2*np.std(times)))

    max_acc = np.max(test_acc)
    iter_max = np.where(test_acc ==  max_acc)

    print('Max accuracy of {:.3f} achieved at epoch {}'.format(max_acc, iter_max[0][0]))
    
    return loss, acc, test_loss, test_acc, idx_f, times, o2

In [36]:
model = ImagesVal2(2, MAX_V, im_size, kernel_size)
loss, acc, test_loss, test_acc, idx_f, times, o2 = evaluating_model3(im_size, model, weighted=0.99)

Epoch 15. Train Loss: 6.489 Accuracy: 0.470 Test Loss: 182.214 Accuracy: 0.196
Average time per epoch 5.938s +- 0.133
Max accuracy of 0.196 achieved at epoch 14


In [37]:
torch.save(model, 'two_channel_weights.pt')